# Sales Forecasting with RasgoQL

This notebook shows how to perform the data preparation and feature engineering for a sales forecasting model. Starting with [AdventureWorks](https://docs.microsoft.com/en-us/sql/samples/adventureworks-install-configure) data loaded in a Snowflake data warehouse, the data will be explored, features created and modeling data extracted.

This analysis will be focused on the internet sales for this company.

## Packages

The documentation for each packaged used in this tutorial is linked below:
* [configparser](https://docs.python.org/3/library/configparser.html)
* [numpy](https://numpy.org/doc/stable/)
* [os](https://docs.python.org/3/library/os.html)
* [pandas](https://pandas.pydata.org/docs/)
* [rasgoQL](https://docs.rasgoql.com/)
* [scikit-learn](https://scikit-learn.org/stable/)
    * [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)
* [XGBoost](https://xgboost.readthedocs.io/en/latest/)

In [1]:
import configparser
import numpy as np
import os
import pandas as pd
import rasgoql
from sklearn.metrics import mean_squared_error
import xgboost as xgb

## Upload AdventureWorks data to Snowflake

See [here](https://medium.com/@griffatrasgo/1946cb9930ea) to add AdventureWorks data to Snowflake.

## Work with RasgoQL

### Get your Snowflake credentials and connect to Snowflake

In [2]:
snowcred = configparser.ConfigParser()
snowcred.read("snowcred.cfg")
account = snowcred['AW']['account']
user = snowcred['AW']['user']
password = snowcred['AW']['password']
role = snowcred['AW']['role']
warehouse = snowcred['AW']['warehouse']
database = snowcred['AW']['database']
schema = snowcred['AW']['schema']

### Connect to Snowflake

In [3]:
creds = rasgoql.SnowflakeCredentials(account=account,
                                     user=user,
                                     password=password,
                                     role=role,
                                     warehouse=warehouse,
                                     database=database,
                                     schema=schema)

In [4]:
rql = rasgoql.connect(credentials=creds)

### Get list of available datasets

List all available tables in the ADVENTUREWORKS database.

In [5]:
rql.list_tables('ADVENTUREWORKS')

,TABLE_NAME,FQTN,TABLE_TYPE,ROW_COUNT,CREATED,LAST_ALTERED
0,ADVENTUREWORKSDWBUILDVERSION,ADVENTUREWORKS.PUBLIC.ADVENTUREWORKSDWBUILDVER...,TABLE,0,2021-11-06 09:07:48.039000-07:00,2022-02-09 00:00:20.470000-08:00
1,DIMACCOUNT,ADVENTUREWORKS.PUBLIC.DIMACCOUNT,TABLE,99,2021-11-06 09:07:48.012000-07:00,2022-02-09 00:00:20.470000-08:00
2,DIMCURRENCY,ADVENTUREWORKS.PUBLIC.DIMCURRENCY,TABLE,105,2021-11-06 09:07:48.071000-07:00,2022-02-09 00:00:20.470000-08:00
3,DIMCUSTOMER,ADVENTUREWORKS.PUBLIC.DIMCUSTOMER,TABLE,18484,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00
4,DIMDATE,ADVENTUREWORKS.PUBLIC.DIMDATE,TABLE,3652,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00
5,DIMDEPARTMENTGROUP,ADVENTUREWORKS.PUBLIC.DIMDEPARTMENTGROUP,TABLE,7,2021-11-06 09:07:48.071000-07:00,2022-02-09 00:00:20.470000-08:00
6,DIMEMPLOYEE,ADVENTUREWORKS.PUBLIC.DIMEMPLOYEE,TABLE,296,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.471000-08:00
7,DIMGEOGRAPHY,ADVENTUREWORKS.PUBLIC.DIMGEOGRAPHY,TABLE,655,2021-11-06 09:07:48.037000-07:00,2022-02-09 00:00:20.471000-08:00
8,DIMORGANIZATION,ADVENTUREWORKS.PUBLIC.DIMORGANIZATION,TABLE,14,2021-11-06 09:07:48.115000-07:00,2022-02-09 00:00:20.470000-08:00
9,DIMPRODUCT,ADVENTUREWORKS.PUBLIC.DIMPRODUCT,TABLE,606,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00


The two tables of interest are ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES containing the sales information and ADVENTUREWORKS.PUBLIC.DIMPROMOTION containing details about any promotions in place.

Examine the transforms that RasgoQL can apply to these tables

In [6]:
rql.list_transforms()

[RasgoTemplate: one_hot_encode(column: column),
 RasgoTemplate: datetrunc(dates: datepart_dict),
 RasgoTemplate: datepart(dates: datepart_dict),
 RasgoTemplate: concat(concat_list: mixed_list, name: value),
 RasgoTemplate: train_test_split(order_by: column_list, train_percent: value),
 RasgoTemplate: standard_scaler(columns_to_scale: column_list, averages: value_list, standarddevs: value_list),
 RasgoTemplate: bin(type: string, bin_count: int, column: column),
 RasgoTemplate: to_date(dates: column_value_dict),
 RasgoTemplate: cast(casts: cast_value_dict),
 RasgoTemplate: label_encode(column: column),
 RasgoTemplate: impute(imputations: imputation_dict, flag_missing_vals: boolean),
 RasgoTemplate: math(math_ops: math_list, names: value_list),
 RasgoTemplate: min_max_scaler(columns_to_scale: column_list, minimums: value_list, maximums: value_list),
 RasgoTemplate: substring(target_col: column, start_pos: value, end_pos: value),
 RasgoTemplate: target_encode(column: column, target: column

### Examine Internet Sales

In [7]:
internet_sales = rql.dataset('ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES')
internet_sales.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07


This looks promising, but I'd like to see a single product sorted by date. This can be done through the use of the filter and order transforms. To use filter, the product we want to filter on is needed, as we don't know that yet, we will just order by *PRODUCTKEY* and *ORDERDATE*.

In [8]:
internet_sales.order(order_by={'PRODUCTKEY':'ASC','ORDERDATE':'ASC'}).preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,214,20121228,20130109,20130104,16313,1,100,8,SO51180,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,214,20121228,20130109,20130104,12132,1,100,7,SO51181,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,214,20121229,20130110,20130105,12390,1,100,8,SO51191,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,214,20121229,20130110,20130105,11241,1,100,7,SO51192,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,214,20121230,20130111,20130106,11338,1,100,8,SO51207,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,214,20121230,20130111,20130106,24604,1,6,9,SO51212,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,214,20121231,20130112,20130107,28204,1,6,9,SO51234,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,214,20121231,20130112,20130107,11615,1,98,10,SO51232,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,214,20121231,20130112,20130107,11061,1,6,9,SO51237,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,214,20121231,20130112,20130107,25625,1,100,8,SO51246,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


This looks reasonable, use this for our modeling. For future reference, what columns exist in this table?

In [9]:
internet_sales.preview().columns.sort_values()

Index(['CARRIERTRACKINGNUMBER', 'CURRENCYKEY', 'CUSTOMERKEY',
       'CUSTOMERPONUMBER', 'DISCOUNTAMOUNT', 'DUEDATE', 'DUEDATEKEY',
       'EXTENDEDAMOUNT', 'FREIGHT', 'ORDERDATE', 'ORDERDATEKEY',
       'ORDERQUANTITY', 'PRODUCTKEY', 'PRODUCTSTANDARDCOST', 'PROMOTIONKEY',
       'REVISIONNUMBER', 'SALESAMOUNT', 'SALESORDERLINENUMBER',
       'SALESORDERNUMBER', 'SALESTERRITORYKEY', 'SHIPDATE', 'SHIPDATEKEY',
       'TAXAMT', 'TOTALPRODUCTCOST', 'UNITPRICE', 'UNITPRICEDISCOUNTPCT'],
      dtype='object')

*PROMOTIONKEY* is probably important for a sales forecast. Promotion information can be found in table ADVENTUREWORKS.PUBLIC.DIMPROMOTION.

In [10]:
promotion = rql.dataset('ADVENTUREWORKS.PUBLIC.DIMPROMOTION')
promotion.preview()

,PROMOTIONKEY,PROMOTIONALTERNATEKEY,ENGLISHPROMOTIONNAME,SPANISHPROMOTIONNAME,FRENCHPROMOTIONNAME,DISCOUNTPCT,ENGLISHPROMOTIONTYPE,SPANISHPROMOTIONTYPE,FRENCHPROMOTIONTYPE,ENGLISHPROMOTIONCATEGORY,SPANISHPROMOTIONCATEGORY,FRENCHPROMOTIONCATEGORY,STARTDATE,ENDDATE,MINQTY,MAXQTY
0,1,1,No Discount,Sin descuento,Aucune remise,0.00,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29,2014-06-30,0,NaN
1,2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),0.02,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,11,14.0
2,3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),0.05,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,15,24.0
3,4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),0.10,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,25,40.0
4,5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),0.15,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,41,60.0
5,6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),0.20,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,61,NaN
6,7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,0.35,Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12,2011-12-28,0,NaN
7,8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,0.10,Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29,2012-01-28,0,NaN
8,9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,0.30,Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29,2012-02-28,0,NaN
9,10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,0.50,Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12,2013-02-26,0,NaN


## Sales Data

Work with the sales and promotion data to create the base modeling time-series features for the sales forecasting model.

### Merge Promo data

First, we want to clean up the promotion data to only keep what needs to be added to the sales data. Drop all columns except *PROMOTIONKEY* and *DISCOUNTPCT* from  promotion using the `drop_columns` transformation.

In [11]:
reduced_promo = promotion.drop_columns(include_cols=['PROMOTIONKEY', 'DISCOUNTPCT'])
reduced_promo.order(order_by={'PROMOTIONKEY':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT
0,1,0.00
1,2,0.02
2,3,0.05
3,4,0.10
4,5,0.15
5,6,0.20
6,7,0.35
7,8,0.10
8,9,0.30
9,10,0.50


Now merge this with the internet sales datausing the `join` transformation.

In [12]:
sales_promo = reduced_promo.join(join_table=internet_sales.fqtn,
                                 join_type='RIGHT',
                                 join_columns={'PROMOTIONKEY':'PROMOTIONKEY'})
sales_promo.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,1,0.0,214,20121228,20130109,20130104,16313,100,8,SO51180,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,1,0.0,214,20121228,20130109,20130104,12132,100,7,SO51181,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,1,0.0,214,20121229,20130110,20130105,12390,100,8,SO51191,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,1,0.0,214,20121229,20130110,20130105,11241,100,7,SO51192,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,1,0.0,214,20121230,20130111,20130106,11338,100,8,SO51207,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,1,0.0,214,20121230,20130111,20130106,24604,6,9,SO51212,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,1,0.0,214,20121231,20130112,20130107,28204,6,9,SO51234,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,1,0.0,214,20121231,20130112,20130107,11615,98,10,SO51232,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,1,0.0,214,20121231,20130112,20130107,11061,6,9,SO51237,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,1,0.0,214,20121231,20130112,20130107,25625,100,8,SO51246,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


### Create Weekly Data

Now, we want to forecast these sales weekly, so we need to extract the week from the *ORDERDATE*. This can be done using the transform `datetrunc`.

In [13]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE':'week'})
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,...,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,ORDERDATE_WEEK
0,1,0.0,214,20121228,20130109,20130104,16313,100,8,SO51180,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04,2012-12-23
1,1,0.0,214,20121228,20130109,20130104,12132,100,7,SO51181,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04,2012-12-23
2,1,0.0,214,20121229,20130110,20130105,12390,100,8,SO51191,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05,2012-12-23
3,1,0.0,214,20121229,20130110,20130105,11241,100,7,SO51192,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05,2012-12-23
4,1,0.0,214,20121230,20130111,20130106,11338,100,8,SO51207,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06,2012-12-30
5,1,0.0,214,20121230,20130111,20130106,24604,6,9,SO51212,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06,2012-12-30
6,1,0.0,214,20121231,20130112,20130107,28204,6,9,SO51234,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
7,1,0.0,214,20121231,20130112,20130107,11615,98,10,SO51232,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
8,1,0.0,214,20121231,20130112,20130107,11061,6,9,SO51237,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
9,1,0.0,214,20121231,20130112,20130107,25625,100,8,SO51246,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30


The new week column is called *ORDERDATE_WEEK*. This is clunky, so let's rename it to *ORDERWEEK* using the `rename` transformation. We could do this by chaining the `rename` transform on the end of the new table *salesds* we created in the previous step.

In [14]:
# newsalesds = salesds.rename(renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
# newsalesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERDATE':'ASC'}).preview()

Alternatively, we can just chain these transformations together.

In [15]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,ORDERWEEK,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,2012-12-23,1,0.0,214,20121228,20130109,20130104,16313,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,2012-12-23,1,0.0,214,20121228,20130109,20130104,12132,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,2012-12-23,1,0.0,214,20121229,20130110,20130105,11241,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,2012-12-23,1,0.0,214,20121229,20130110,20130105,12390,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,2012-12-30,1,0.0,214,20121230,20130111,20130106,24604,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,2012-12-30,1,0.0,214,20121230,20130111,20130106,11338,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,2012-12-30,1,0.0,214,20121231,20130112,20130107,11061,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,2012-12-30,1,0.0,214,20121231,20130112,20130107,25625,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,2012-12-30,1,0.0,214,20121231,20130112,20130107,11615,98,10,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,2012-12-30,1,0.0,214,20121231,20130112,20130107,28204,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


Now we can aggregate this to the product-week level and create aggregations of the *'DISCOUNTAMOUNT'*, *'DISCOUNTPCT'*, *'ORDERQUANTITY'*, *'PRODUCTSTANDARDCOST'*, *'SALESAMOUNT'*, *'TAXAMT'*, *'TOTALPRODUCTCOST'*, *'UNITPRICE'*, *'UNITPRICEDISCOUNTPCT'* using the `aggregate` transform.

In [16]:
salesds_agg = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN','MAX','AVG','SUM'], 
                                              'DISCOUNTPCT': ['MIN','MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']})
salesds_agg.describe().to_df()

,FEATURE,DTYPE,COUNT,NULL_COUNT,UNIQUE_COUNT,MOST_FREQUENT,MEAN,STD_DEV,MIN,_25_PERCENTILE,_50_PERCENTILE,_75_PERCENTILE,MAX
0,DISCOUNTPCT_MAX,FLOAT,6935,0,4,0,0.002531,0.012647,0,0.0000,0.0000,0.0000,0.2
1,TOTALPRODUCTCOST_AVG,NUMBER,6935,0,45,1481.9379000000,661.195888,651.412800,0.8565000000,26.1763,461.4448,1251.9813,2171.2942000000
2,ORDERQUANTITY_SUM,NUMBER,6935,0,97,1,8.709156,13.494000,1,2.0000,4.0000,8.0000,105
3,TAXAMT_SUM,NUMBER,6935,0,772,62.6392,338.672564,502.119733,0.3992,41.9832,118.7760,408.2376,3721.4008
4,PRODUCTSTANDARDCOST_AVG,NUMBER,6935,0,45,486.7066000000,661.195888,651.412800,0.8565000000,26.1763,461.4448,1251.9813,2171.2942000000
5,UNITPRICEDISCOUNTPCT_MIN,FLOAT,6935,0,1,0,0.000000,0.000000,0,0.0000,0.0000,0.0000,0
6,ORDERWEEK,TIMESTAMP_NTZ,6935,0,162,2013-06-02 00:00:00.000,NaN,NaN,2010-12-26 00:00:00.000,NaN,NaN,NaN,2014-01-26 00:00:00.000
7,DISCOUNTAMOUNT_AVG,FLOAT,6935,0,1,0,0.000000,0.000000,0,0.0000,0.0000,0.0000,0
8,SALESAMOUNT_SUM,NUMBER,6935,0,772,782.9900,4233.406953,6276.496610,4.9900,524.7900,1484.7000,5102.9700,46517.5100
9,UNITPRICEDISCOUNTPCT_SUM,FLOAT,6935,0,1,0,0.000000,0.000000,0,0.0000,0.0000,0.0000,0


This gives us statistics for each product over a given week.

### Time-series feature engineering

For sales forcasting, in addition to the lagged variables, we need to know what the sales were in prior weeks. The transform `lag` can create these variables for us. In this case we will lag the following variables *'DISCOUNTAMOUNT_AVG'*, *'DISCOUNTPCT_AVG'*, *'ORDERQUANTITY_SUM'*, *'PRODUCTSTANDARDCOST_AVG'*, *'SALESAMOUNT_SUM'*, *'TAXAMT_SUM'*, *'TOTALPRODUCTCOST_SUM'*,*'UNITPRICEDISCOUNTPCT_AVG'*, *'UNITPRICE_AVG'*, *'UNITPRICE_SUM'*
over *1*, *2*, *3*, and *12* weeks.

In [17]:
salesds = salesds_agg.lag(columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1, 2, 3, 12],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
   
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICEDISCOUNTPCT_AVG_3,LAG_UNITPRICEDISCOUNTPCT_AVG_12,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,None,None,None,None,None,None,None,None
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,None,None,None,139.9600,None,None,None
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None


In addition to lag variables, the moving average of the quantites can be useful. In this case, we'll calculate the moving average over *4* observations of *ORDERQUANTITY_SUM* and *TOTALPRODUCTCOST_SUM* using the transform `moving_avg`.

In [18]:
salesds = salesds.moving_avg(input_columns=['ORDERQUANTITY_SUM', 'SALESAMOUNT_SUM'],
                             window_sizes=[4],
                             order_by=['PRODUCTKEY', 'ORDERWEEK'],
                             partition=['PRODUCTKEY'])
    
salesds.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,4.000,139.9600000
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,None,None,None,139.9600,None,None,None,7.000,244.9300000
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000


#### Save result

At this point, the data has been aggregated to weekly data and multiple transformations have been applied. This could be a good starting point for additional analysis and useful for visualization. For this reason, we will publish it back to the data warehouse to make it available for others to use. This can be done with the `save` function.  *table_type* can be either *TABLE* or *VIEW* depending on the type of object you want to create.

In [19]:
weeklysales = salesds.save(table_name="WKSP_AW_WEEKLY_SALES",
                           table_type="VIEW")
weeklysales

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_AW_WEEKLY_SALES, type=view)

Using this dataset, we can continue data preparation.

### Create Modeling Data

We can now begin final preparation for modeling with this dataset. To do this, we need to do three things. First, the target (next weeks sales needs to be created). Second, the categorical variables should be one-hot encoded. Finally, missing values should be imputed for the numeric columns.

#### Target Creation

Use the `lag` transform with a negative lag value to get next weeks sales as the target. While doing this, rename the value to make it clear that it is the target.

In [20]:
modelingds = weeklysales.lag(columns=['SALESAMOUNT_SUM'],
                             amounts=[-1],
                             order_by=['PRODUCTKEY', 'ORDERWEEK'],
                             partition =['PRODUCTKEY']).rename(
                             renames={'LAG_SALESAMOUNT_SUM__1': 'TARGET_SALESAMOUNT'})
modelingds.preview().columns.sort_values()

Index(['DISCOUNTAMOUNT_AVG', 'DISCOUNTAMOUNT_MAX', 'DISCOUNTAMOUNT_MIN',
       'DISCOUNTAMOUNT_SUM', 'DISCOUNTPCT_AVG', 'DISCOUNTPCT_MAX',
       'DISCOUNTPCT_MIN', 'DISCOUNTPCT_SUM', 'LAG_DISCOUNTAMOUNT_AVG_1',
       'LAG_DISCOUNTAMOUNT_AVG_12', 'LAG_DISCOUNTAMOUNT_AVG_2',
       'LAG_DISCOUNTAMOUNT_AVG_3', 'LAG_DISCOUNTPCT_AVG_1',
       'LAG_DISCOUNTPCT_AVG_12', 'LAG_DISCOUNTPCT_AVG_2',
       'LAG_DISCOUNTPCT_AVG_3', 'LAG_ORDERQUANTITY_SUM_1',
       'LAG_ORDERQUANTITY_SUM_12', 'LAG_ORDERQUANTITY_SUM_2',
       'LAG_ORDERQUANTITY_SUM_3', 'LAG_PRODUCTSTANDARDCOST_AVG_1',
       'LAG_PRODUCTSTANDARDCOST_AVG_12', 'LAG_PRODUCTSTANDARDCOST_AVG_2',
       'LAG_PRODUCTSTANDARDCOST_AVG_3', 'LAG_SALESAMOUNT_SUM_1',
       'LAG_SALESAMOUNT_SUM_12', 'LAG_SALESAMOUNT_SUM_2',
       'LAG_SALESAMOUNT_SUM_3', 'LAG_TAXAMT_SUM_1', 'LAG_TAXAMT_SUM_12',
       'LAG_TAXAMT_SUM_2', 'LAG_TAXAMT_SUM_3', 'LAG_TOTALPRODUCTCOST_SUM_1',
       'LAG_TOTALPRODUCTCOST_SUM_12', 'LAG_TOTALPRODUCTCOST_SUM_2',
  

#### Categorical encoding

The only column that needs to be encoded *PRODUCTKEY*. We can use the `one_hot_encode`, `target_encode` or the `label_encode` transforms. 

For features with a large number of categorties and when using tree-based modeling algorithms, the `label_encode` transform is a useful technique to encode categorical variables. The `target_encode` transform replaces the category by the mean target value of that category. Target encoding is a very powerful techinque to encode these high-cardinality categorical variables efficiently and help improve model performance. In this case, we will target encode the product_key

In [21]:
modelingds_encoded = modelingds.target_encode(column='PRODUCTKEY',
                                      target='TARGET_SALESAMOUNT')

modelingds_encoded.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,TARGET_SALESAMOUNT,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,...,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,PRODUCTKEY_TARGET_ENCODED
0,349.9000,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,4.000,139.9600000,1366.452
1,419.8800,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,139.9600,None,None,None,7.000,244.9300000,1366.452
2,209.9400,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666,1366.452
3,384.8900,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000,1366.452
4,1084.6900,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000,1366.452
5,1539.5600,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000,1366.452
6,1189.6600,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000,1366.452
7,1154.6700,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000,1366.452
8,1294.6300,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000,1366.452
9,1014.7100,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000,1366.452


#### Imputation

As a final step before modeling, all numeric columns should have missing values replaced by a number. This can be done by the `impute` transformation. If a linear or logistic regression, SVM or neural network algorithm was going to be applied, we may want to impute the mean or median. This could be done by passing 'mean' or 'median' in through the imputations dictionary.

As the modeling algoritm applied here is tree-based, we can simply impute and extreme value. All of the features created are non-negative or close to zero, so we will impute a very large negative number, *-999,999*.

In [22]:
imputation_dict = {'DISCOUNTAMOUNT_AVG': -999999,
                   'DISCOUNTAMOUNT_MAX': -999999,
                   'DISCOUNTAMOUNT_MIN': -999999,
                   'DISCOUNTAMOUNT_SUM': -999999,
                   'DISCOUNTPCT_AVG': -999999,
                   'DISCOUNTPCT_MAX': -999999,
                   'DISCOUNTPCT_MIN': -999999,
                   'DISCOUNTPCT_SUM': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_1': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_12': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_2': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_3': -999999,
                   'LAG_DISCOUNTPCT_AVG_1': -999999,
                   'LAG_DISCOUNTPCT_AVG_12': -999999,
                   'LAG_DISCOUNTPCT_AVG_2': -999999,
                   'LAG_DISCOUNTPCT_AVG_3': -999999,
                   'LAG_ORDERQUANTITY_SUM_1': -999999,
                   'LAG_ORDERQUANTITY_SUM_12': -999999,
                   'LAG_ORDERQUANTITY_SUM_2': -999999,
                   'LAG_ORDERQUANTITY_SUM_3': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_1': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_12': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_2': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_3': -999999,
                   'LAG_SALESAMOUNT_SUM_1': -999999,
                   'LAG_SALESAMOUNT_SUM_12': -999999,
                   'LAG_SALESAMOUNT_SUM_2': -999999,
                   'LAG_SALESAMOUNT_SUM_3': -999999,
                   'LAG_TAXAMT_SUM_1': -999999,
                   'LAG_TAXAMT_SUM_12': -999999,
                   'LAG_TAXAMT_SUM_2': -999999,
                   'LAG_TAXAMT_SUM_3': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_1': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_12': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_2': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_3': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_1': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_12': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_2': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_3': -999999,
                   'LAG_UNITPRICE_AVG_1': -999999,
                   'LAG_UNITPRICE_AVG_12': -999999,
                   'LAG_UNITPRICE_AVG_2': -999999,
                   'LAG_UNITPRICE_AVG_3': -999999,
                   'LAG_UNITPRICE_SUM_1': -999999,
                   'LAG_UNITPRICE_SUM_12': -999999,
                   'LAG_UNITPRICE_SUM_2': -999999,
                   'LAG_UNITPRICE_SUM_3': -999999,
                   'MEAN_ORDERQUANTITY_SUM_4': -999999,
                   'MEAN_SALESAMOUNT_SUM_4': -999999,
                   'ORDERQUANTITY_SUM': -999999,
                   'PRODUCTSTANDARDCOST_AVG': -999999,
                   'PRODUCTSTANDARDCOST_SUM': -999999,
                   'SALESAMOUNT_SUM': -999999,
                   'TAXAMT_SUM': -999999,
                   'TOTALPRODUCTCOST_AVG': -999999,
                   'TOTALPRODUCTCOST_SUM': -999999,
                   'UNITPRICEDISCOUNTPCT_AVG': -999999,
                   'UNITPRICEDISCOUNTPCT_MAX': -999999,
                   'UNITPRICEDISCOUNTPCT_MIN': -999999,
                   'UNITPRICEDISCOUNTPCT_SUM': -999999,
                   'UNITPRICE_AVG': -999999,
                   'UNITPRICE_SUM': -999999}

In [23]:
modelingds_imputed = modelingds_encoded.impute(imputations=imputation_dict)

modelingds_imputed.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,TARGET_SALESAMOUNT,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,...,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,PRODUCTKEY_TARGET_ENCODED
0,349.9000,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999999.0000000000,-999999.0000000000,-999999.0000000000,-999999.0000,-999999.0000,-999999.0000,-999999.0000,4.000,139.9600000,1366.452
1,419.8800,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999999.0000000000,-999999.0000000000,-999999.0000000000,139.9600,-999999.0000,-999999.0000,-999999.0000,7.000,244.9300000,1366.452
2,209.9400,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,-999999.0000000000,349.9000,139.9600,-999999.0000,-999999.0000,8.666,303.2466666,1366.452
3,384.8900,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,419.8800,349.9000,139.9600,-999999.0000,8.000,279.9200000,1366.452
4,1084.6900,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,209.9400,419.8800,349.9000,-999999.0000,9.750,341.1525000,1366.452
5,1539.5600,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,384.8900,209.9400,419.8800,-999999.0000,15.000,524.8500000,1366.452
6,1189.6600,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,1084.6900,384.8900,209.9400,-999999.0000,23.000,804.7700000,1366.452
7,1154.6700,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,1539.5600,1084.6900,384.8900,-999999.0000,30.000,1049.7000000,1366.452
8,1294.6300,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,1189.6600,1539.5600,1084.6900,-999999.0000,35.500,1242.1450000,1366.452
9,1014.7100,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,-999999.0000000000,1154.6700,1189.6600,1539.5600,-999999.0000,37.000,1294.6300000,1366.452


#### Train-test split

As this is a time-series problem, a random train-test split won't work as there will be leakage from observations near the end of the time frame in the training set to observations earlier than this in the test set. The way to avoid this problem is to perform the split based on the date. The transformation `train_test_split` can do this by passing the date columns through the parameter **order_by**.

In [24]:
modelingds_split = modelingds_imputed.train_test_split(order_by=['ORDERWEEK'], 
                                         train_percent=0.8)
    
modelingds_split.order(order_by={'PRODUCTKEY':'ASC', 'ORDERWEEK':'ASC'}).preview()

,TARGET_SALESAMOUNT,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,...,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,PRODUCTKEY_TARGET_ENCODED,TT_SPLIT
0,349.9000,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999999.0000000000,-999999.0000000000,-999999.0000,-999999.0000,-999999.0000,-999999.0000,4.000,139.9600000,1366.452,TRAIN
1,419.8800,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999999.0000000000,-999999.0000000000,139.9600,-999999.0000,-999999.0000,-999999.0000,7.000,244.9300000,1366.452,TRAIN
2,209.9400,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-999999.0000000000,-999999.0000000000,349.9000,139.9600,-999999.0000,-999999.0000,8.666,303.2466666,1366.452,TRAIN
3,384.8900,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,419.8800,349.9000,139.9600,-999999.0000,8.000,279.9200000,1366.452,TRAIN
4,1084.6900,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,209.9400,419.8800,349.9000,-999999.0000,9.750,341.1525000,1366.452,TRAIN
5,1539.5600,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,384.8900,209.9400,419.8800,-999999.0000,15.000,524.8500000,1366.452,TRAIN
6,1189.6600,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,1084.6900,384.8900,209.9400,-999999.0000,23.000,804.7700000,1366.452,TRAIN
7,1154.6700,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,1539.5600,1084.6900,384.8900,-999999.0000,30.000,1049.7000000,1366.452,TRAIN
8,1294.6300,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,1189.6600,1539.5600,1084.6900,-999999.0000,35.500,1242.1450000,1366.452,TRAIN
9,1014.7100,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,-999999.0000000000,1154.6700,1189.6600,1539.5600,-999999.0000,37.000,1294.6300000,1366.452,TRAIN


#### Save Modeling Dataset

We can now save this modeling dataset so we can return to it in the future.

In [25]:
modeling = modelingds_split.save(table_name="WKSP_SALES_FORECASTING",
                                 table_type="VIEW")
modeling

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_SALES_FORECASTING, type=view)

Grab the table name for production use.

In [26]:
modeling_fqtn = modeling.fqtn

### Modeling

We are now ready to build the model. First, get the modeling data from Rasgo using `to_df`.

In [27]:
df = modeling.to_df().reset_index(drop=True)

Check for numeric datatypes and convert the numeric ones to floats.

In [28]:
for c in df.select_dtypes(exclude=[np.number]).columns:
    if c not in ['ORDERWEEK', 'TT_SPLIT']:
        df[c] = pd.to_numeric(df[c])

Eliminate the last week of data as there is no target.

In [29]:
df = df[~df.TARGET_SALESAMOUNT.isna()]

#### Train the model

First, split the data using the TT_SPLIT column.

In [30]:
df_train = df[df['TT_SPLIT'] == 'TRAIN'].drop(columns=['TT_SPLIT', 'ORDERWEEK'])
df_test = df[df['TT_SPLIT'] == 'TEST'].drop(columns=['TT_SPLIT', 'ORDERWEEK'])

In [31]:
y_train = df_train['TARGET_SALESAMOUNT']
X_train = df_train.drop(columns=['TARGET_SALESAMOUNT'])
y_test = df_test['TARGET_SALESAMOUNT']
X_test = df_test.drop(columns=['TARGET_SALESAMOUNT'])

#### Fit the model

For illustration purposes, we are just fitting the model with a single set of parameters. In general, you should optimize the hyperparameters before building the final model. That process is beyond the scope of this document.

In [32]:
model = xgb.XGBRegressor(n_estimators= 100,
                         max_depth=5,
                         eta=0.01,
                         random_state=1066,
                         subsample=0.7,
                         colsample_bytree=0.7)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.01, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.00999999978, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=1066,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Check the performance

In [33]:
model.predict(X_test)

array([ 112.69703 ,  268.66727 , 2374.9377  , ...,  203.9415  ,
        110.232765,  113.94583 ], dtype=float32)

In [34]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

4846.567085550658

### Predict on new data

Since our feature engineering was saved in Rasgo, as new data enters the system, it will automatically be prepared for modeling. We can just pull the data in question and make a prediction on it.

In this case, if we are making these predictions each week, we can just pull the most recent week. In this particular data, that is '*2014-01-19*'.

#### Pull the data

Use to_df to grab the data from this date. We have several columns not needed in the model, so we will drop those as well.

In [35]:
predictdf = rql.dataset(modeling_fqtn).filter(filter_statements=["ORDERWEEK = '2014-01-19'"]).to_df()
for c in predictdf.select_dtypes(exclude=[np.number]).columns:
    if c not in ['ORDERWEEK', 'TT_SPLIT']:
        predictdf[c] = pd.to_numeric(predictdf[c])
predictdf.head()

,TARGET_SALESAMOUNT,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,...,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,PRODUCTKEY_TARGET_ENCODED,TT_SPLIT
0,99.98,234,2014-01-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.99,49.99,449.91,249.95,49.99,299.94,4.50,224.9550,393.781,TEST
1,NaN,479,2014-01-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.99,8.99,26.97,17.98,26.97,386.57,2.50,22.4750,274.195,TEST
2,236.39,538,2014-01-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.49,21.49,322.35,236.39,236.39,494.27,13.25,284.7425,393.229,TEST
3,209.97,474,2014-01-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.99,69.99,279.96,559.92,629.91,419.94,7.25,507.4275,407.826,TEST
4,139.98,475,2014-01-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.99,69.99,419.94,139.98,489.93,629.91,4.75,332.4525,463.519,TEST


Now use the model to get the sales forecast. We will create a dataframe to hold the predictions then drop the columns not needed by the model before making the prediction.

In [36]:
salesforecastdf = predictdf[['PRODUCTKEY', 'ORDERWEEK']].copy()
salesforecastdf['forecast'] = model.predict(predictdf.drop(columns=['TT_SPLIT', 'ORDERWEEK', 'TARGET_SALESAMOUNT']))
salesforecastdf

,PRODUCTKEY,ORDERWEEK,forecast
0,234,2014-01-19,181.138901
1,479,2014-01-19,112.997650
2,538,2014-01-19,207.630569
3,474,2014-01-19,281.378265
4,475,2014-01-19,265.333160
5,481,2014-01-19,110.232765
6,467,2014-01-19,111.407784
7,529,2014-01-19,111.407784
8,463,2014-01-19,114.299706
9,222,2014-01-19,711.570129
